Proof. Let $|G\rangle=|0\rangle_{a_1}|0\rangle_{a_2}|0\rangle_{a_3} \equiv|0\rangle_a$ be a computational basis state, and let $F_j=\{f(j, l)\}_{l \in[d]}$ be the set of column indices to all non-zero elements in row $j$. Ref. [4] shows how each of the isometries
$$
\begin{aligned}
& \hat{T}_1=\sum_j\left|\psi_j\right\rangle\left\langle\left.\left. 0\right|_a\left\langle\left. j\right|_s, \mid \psi_j\right\rangle=\sum_{p \in F_j} \frac{|p\rangle_{a_3}}{\sqrt{d}}\left(\sqrt{\frac{H_{p j}}{\|\hat{H}\|_{\max }}}|0\rangle_{a_1}+\sqrt{1-\frac{\left|H_{p j}\right|}{\|\hat{H}\|_{\max }}}|1\rangle_{a_1}\right) \right\rvert\, 0\right\rangle_{a_2}|j\rangle_s, \\
& \hat{T}_2=\sum_k\left|\chi_k\right\rangle\left\langle0 | _ { a } \left\langle\left. k\right|_s,\left\langle\chi_k\right|=\sum_{p \in F_k} \frac{\left\langle\left. p\right|_s\right.}{\sqrt{d}}\left(\sqrt { \frac { H _ { k p } } { \| \hat { H } \| _ { \operatorname { m a x } } } } \left\langle\left.0\right|_{a_2}+\sqrt{1-\frac{\left|H_{k p}\right|}{\|\hat{H}\|_{\max }}}\left\langle\left. 1\right|_{a_2}\right)\left\langle0 | _ { a _ { 1 } } \left\langle\left. k\right|_{a_3},\right.\right.\right.\right.\right.\right.
\end{aligned}
$$
can be implemented using using 2 queries to $\hat{O}_H$ and 1 query to $\hat{O}_F$. By construction, the overlap of these states is $\left\langle\chi_k \mid \psi_j\right\rangle=\frac{H_{k j}}{d\|\hat{H}\|_{\max }}$. Now choose $\hat{U}=\hat{T}_2^{\dagger} \hat{T}_1$. By a direct computation,
$$
\left(\left\langle\left. G\right|_a \otimes \hat{\mathcal{I}}_s\right) \hat{U}\left(|G\rangle_a \otimes \hat{\mathcal{I}}_s\right)=\sum_{k, j}|k\rangle_s\left\langle\chi_k \mid \psi_j\right\rangle\left\langle\left.\left. j\right|_s=\sum_{k, j} \frac{H_{k j}}{d\|\hat{H}\|_{\max }} \right\rvert\, k\right\rangle\left\langle\left. j\right|_s=\frac{\hat{H}}{d\|\hat{H}\|_{\max }} .\right.\right.
$$

来源于Hamiltonian Simulation by Qubitization第八页，3.2 d-Sparse Hamiltonians，Lemma 6 (Standard-Form Encoding of a d-Sparse Hamiltonian)

In [4]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator


# Construct the isometries T1 and T2
def construct_T1(H,H_max_norm):
    N = H.shape[0]
    d = N
    T1 = np.zeros((4*N*d, 4*N*d), dtype=complex)
    for j in range(N):
        F_j = np.nonzero(H[j, :])[0]
        for p in F_j:
            T1[ j,(0+ 0*2+ p*4)*d + j] = np.sqrt(H[p, j] / (d * H_max_norm))
            T1[ j,(0+ 0*2+ p*4)*d + j] = np.sqrt(1 - np.abs(H[p, j])/H_max_norm) / np.sqrt(d)
    return T1

def construct_T2(H, H_max_norm):
    N = H.shape[0]
    d = N 
    T2 = np.zeros((4*N*d, 4*N*d), dtype=complex)
    for k in range(N):
        F_k = np.nonzero(H[k, :])[0]
        for p in F_k:
            T2[k, (0+ 0*2+ k*4)*d + p] = np.sqrt(H[k, p] / (d * H_max_norm))
            T2[k, (0+ 1*2+ k*4)*d + p] = np.sqrt(1 - np.abs(H[p, k])/H_max_norm) / np.sqrt(d)
    return T2

# Define the Hermitian matrix H
H = np.array([[1, 0.5], [0.5, 1]], dtype=complex)
N = H.shape[0]
n = int(np.log2(N))
# Compute the maximum absolute value of the elements in H
H_max_norm = np.max(np.abs(H))
# Construct the unitary operator U
T1 = construct_T1(H,H_max_norm)
T2 = construct_T2(H,H_max_norm)
U = np.conj(T2).T@ T1

# Convert the unitary operator U to a quantum circuit
def unitary_to_circuit(U):
    qc = QuantumCircuit(n)
    qc.unitary(Operator(U), range(n))
    return qc

# Verify the unitary encoding
def verify_encoding(U, H, d, H_max_norm):
    encoded_matrix = U / (d * H_max_norm)
    print("Encoded Matrix:\n", encoded_matrix)
    print("\nOriginal Matrix H / (d * ||H||_max):\n", H / (d * H_max_norm))

# Convert the unitary operator U to a quantum circuit
qc = unitary_to_circuit(U)

# Print the quantum circuit
print("Quantum Circuit:")
print(qc)

# Verify the unitary encoding
verify_encoding(U, H, N, H_max_norm)


ValueError: Input matrix is not unitary.

Theorem 4.1. Let $c(j, \ell)$ be a function that gives the row index of the $\ell$ th (among a list of $s$ ) non-zero matrix elements in the jth column of an $s$-sparse matrix $A \in \mathbb{C}^{N \times N}$ with $N=2^n$, where $s=2^m$. If there exists a unitary $O_c$ such that
$$
O_c|\ell\rangle|j\rangle=|\ell\rangle|c(j, \ell)\rangle
$$
and a unitary $O_A$ such that
$$
O_A|0\rangle|\ell\rangle|j\rangle=\left(A_{c(j, \ell), j}|0\rangle+\sqrt{1-\left|A_{c(j, \ell), j}\right|^2}|1\rangle\right)|\ell\rangle|j\rangle,
$$
then
$$
U_A=\left(I_2 \otimes D_s \otimes I_N\right)\left(I_2 \otimes O_c\right) O_A\left(I_2 \otimes D_s \otimes I_N\right)
$$
block encodes $A / s$. Here $D_s$ is called a diffusion operator and is defined as
$$
D_s \equiv \underbrace{H \otimes H \otimes \cdots \otimes H}_m
$$

来源于Quantum Circuits for Block Encodings of Sparse Matrices 第七页 Theorem 4.1.

In [11]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
# Define the dimensions and sparsity
n = 1  # N = 2^n, for N = 8, n = 3
m = 1  # s = 2^m, for s = 4, m = 2
N = 2**n
s = 2**m

# Example Hermitian matrix H
H = np.array([[1, 0.5], [-0.5, 1]], dtype=complex)

# Function c(j, ell) - Determines the row index of the ell-th non-zero element in the j-th column
def c(j, ell, H):
    # Identify the non-zero elements in the j-th column
    non_zero_indices = np.nonzero(H[:, j])[0]
    return non_zero_indices[ell]

# Create the diffusion operator D_s
def create_diffusion_operator(m):
    qc = QuantumCircuit(m)
    for i in range(m):
        qc.h(i)
    return qc.to_gate(label="D_s")

# Create the unitary O_c
# Create the unitary O_c
def create_O_c(n, m, H):
    size = 2**(n + m)
    O_c_matrix = np.zeros((size,size), dtype=complex)

    for ell in range(2**m):
        for j in range(2**n):
            row_index = c(j, ell, H)
            O_c_matrix[ell+ j*(2**m), ell  + row_index* (2**m)] = 1

    print(O_c_matrix)
    qc = QuantumCircuit(n+m)
    qc.unitary(O_c_matrix, range(n + m))
    return qc.to_gate()

# Create the unitary O_A
def create_O_A(n, m, H):
    size = 2**(1 + n + m)
    O_A_matrix = np.zeros((size,size), dtype=complex)

    for ell in range(2**m):
        for j in range(2**n):
            row_index = c(j, ell, H)
            A_value = H[row_index, j]
            idx = (ell * (2**n) + j) * 2
            O_A_matrix[idx, idx] = A_value
            O_A_matrix[idx, idx + 1] = np.sqrt(1 - np.abs(A_value)**2)
    qc = QuantumCircuit(1+n+m)
    qc.unitary(O_A_matrix, range(1+n + m))
    return qc.to_gate()

# Combine the unitaries to create U_A
def create_U_A(n, m, H):
    D_s = create_diffusion_operator(m)
    O_c = create_O_c(n, m, H)
    O_A = create_O_A(n, m, H)

    qc = QuantumCircuit(1 + n + m)
    qc.append(D_s, range(1,m+1))
    qc.append(O_c, range(m, m + n))
    qc.append(O_A, range(1 + n + m))
    qc.append(D_s, range(1,m+1))
    return qc

# Set the number of qubits
n_qubits = 1 + n + m
qc = create_U_A(n, m, H)

# Get the unitary matrix of the circuit
unitary = Operator(qc).data

# Extract the top-left block
top_left_block = unitary[:N, :N]

# Calculate the expected block encoding
expected_block = H / s

# Print the results
print("Top-left block of the unitary matrix U_A:\n", top_left_block)
print("\nExpected block encoding H/s:\n", expected_block)

# Verify the block encoding
if np.allclose(top_left_block, expected_block, atol=1e-2):
    print("\nThe circuit correctly block encodes the Hermitian matrix H.")
else:
    print("\nThe circuit does not correctly block encode the Hermitian matrix H.")


[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]


ValueError: Input matrix is not unitary.